# 1. Transformer와 비교해 변경사항 정리

## 1-1) 모델 구조 비교

![](transformer.jpg)

![](gpt.JPG)

## 1-2) 변경사항 정리

1. GPT-1 은 Transformer의 Decoder 부분만 사용
2. encoder의 정보를 가져오는 Multi-Head attention 부분은 사용하지 않음
3. positional encoding을 사용하지 않고, positinal embeding을 사용함
4. 강화학습으로 pretrained 된 모델을 finetunning 하는 학습과정 갖음

# 2. 데이터 불러오기 & 전처리

## 2-1) 데이터 불러오기

In [48]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [49]:
file_path = "~/aiffel/transformer_chatbot/data/ChatbotData.csv"
file_path = os.path.expanduser(file_path)
data = pd.read_csv(file_path)
data = data[['Q','A']]
data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [50]:
data.shape

(11823, 2)

## 2-2) 데이터 전처리

In [51]:
#한국어 데이터 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower() #한국어에 왜 쓰는거지?

  # 특수 문자 제거
  sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z?.!,]+", " ", sentence)
    
  # 여러 개의 공백을 하나로 합침
  sentence = re.sub(r'[" "]+', " ", sentence)

  sentence = sentence.strip()
  return sentence

In [52]:
### Q, A 데이터 전처리
questions = []
answers = []
for [i, j] in zip(data['Q'], data['A']):
    questions.append(preprocess_sentence(i))
    answers.append(preprocess_sentence(j))

In [53]:
print(len(questions))
print(len(answers))

11823
11823


## 2-3) 병렬 데이터 전처리하기


### (1) 단어장(Vocabulary) 만들기¶

In [54]:
import tensorflow_datasets as tfds

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [55]:
# 전체 vocabulary 리스트 얻기
vocab_list = tokenizer.subwords 

# vocabulary 리스트 출력 
print("vocab_size = ",tokenizer.vocab_size)
print("vocab_lise_size = ",len(vocab_list))

#결과에 차이가 있는 이유
#UNK, BOS, EOS, Padding 토큰과 같은 특수 토큰
#SubwordTextEncoder는 내부적으로 몇몇 토큰들을 예약(reserved)해둠

vocab_size =  8168
vocab_lise_size =  7911


In [56]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8168]
END_TOKEN의 번호 : [8169]


In [57]:
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8170


### (2) 각 단어를 고유한 정수로 인코딩(Integer encoding) & 패딩(Padding)

In [58]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5756, 607, 2488, 4159]
정수 인코딩 후의 21번째 답변 샘플: [2354, 7507, 5, 6270, 94, 7958]


In [59]:
#문장 길이
MAX_LENGTH = 40

In [60]:
#패딩추가 패키지
from tensorflow.keras.preprocessing.sequence import pad_sequences

#정수 인코딩
#최대 길이 초과 샘플 제거 및 패딩
def tokenize_and_filter(inputs, outputs):
    #토큰화 input, out 담을 그릇
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sen1, sen2) in zip(inputs, outputs):
    #정수 인코딩 및 시작 토큰과 종료 토큰 추가
    
        #문장의 시작과 끝에 시작, 종료 토큰 추가
        sen1 = START_TOKEN + tokenizer.encode(sen1) + END_TOKEN
        sen2 = START_TOKEN + tokenizer.encode(sen2) + END_TOKEN
        
        #문장 길이가 40 이상인 문장만 추가
        if len(sen1) <= MAX_LENGTH and len(sen2) <= MAX_LENGTH:
            tokenized_inputs.append(sen1)
            tokenized_outputs.append(sen2)
    
    tokenized_inputs = pad_sequences(
        tokenized_inputs,
        maxlen = MAX_LENGTH,
        padding = "post", #시퀀스의 끝 부분에 패딩 추가
    ) 
    
    tokenized_outputs = pad_sequences(
        tokenized_outputs,
        maxlen = MAX_LENGTH,
        padding = "post", #시퀀스의 끝 부분에 패딩 추가
    ) 
    
    return tokenized_inputs, tokenized_outputs 

In [61]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8170
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


### (3) 교사 강요(Teacher Forcing) 사용

### Supervised learning Dataset

In [62]:
#tf.data.Dataset API의 입력으로 사용하여 파이프라인을 구성

BATCH_SIZE = 64
BUFFER_SIZE = 20000


#from_tensor.slices : numpy나 텐서 slice 후 객체생성
dataset = tf.data.Dataset.from_tensor_slices([ 
        answers[:,:-1],
        #답변의 마지막 토큰을 제외한 부분을 입력으로 사용
        answers[:,1:]
        #답변 데이터에서 첫 번째 토큰을 제외한 부분을 레이블(정답)로 사용
    ])

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### Unsupervised learning Dataset

# 3. 모델 구현

## 3-1) scaled_dot_product_attention

In [63]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    #Query와 K의 점곱
    matmul_qk = tf.matmul(query, key, transpose_b = True)
    
    #임베딩 차원을 가져와 실수 
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth) #임베딩차원의 제곱근으로 나눠줌
    
    # 패딩에 마스크 추가
    # 패딩인 부분은 점수계산에 팜여하지 안도록 함
    if mask is not None:
        logits += (mask * -1e9)
    
    #soft max
    attention_weights = tf.nn.softmax(logits, axis=-1)
    
    # 최종 어텐션 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    
    return output

## 3-2) MultiHeadAttention

In [64]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__ (self, d_model, num_heads, name="multi_head_attention"):
        
        #부모 클래스 생성자 호출, 이름을 name으로 설정
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads #나누는 헤드 개수
        self.d_model = d_model #임베딩 차원
        
        #임베딩 차원이 헤드와 나누어 떨어지는지 확인
        #만족하지 않을 시 에러
        assert d_model % self.num_heads == 0
        
        #각 헤드의 차원 계산
        self.depth = d_model // self.num_heads
        
        #각각 쿼리(Query), 키(Key), 값(Value) 벡터를 생성하기 위한 Dense 레이어
        #가중치 행렬을 곱하고 편향을 더하여 출력 벡터를 생성하는 선형 변환 레이어
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        
        #어텐션 연산 후 최종 결과를 생성하기 위한 Dense 레이어
        self.dense = tf.keras.layers.Dense(units=d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs,
            shape = (batch_size,-1,self.num_heads,self.depth)
        )
        return tf.transpose(inputs, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

         # Q, K, V에 각각 Dense를 적용합니다.
        query = self.query_dense(query)  # (batch_size, seq_len, d_model)
        key = self.key_dense(key)        # (batch_size, seq_len, d_model)
        value = self.value_dense(value)  # (batch_size, seq_len, d_model)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다.
        query = self.split_heads(query, batch_size)  # (batch_size, num_heads, seq_len, depth)
        key = self.split_heads(key, batch_size)      # (batch_size, num_heads, seq_len, depth)
        value = self.split_heads(value, batch_size)  # (batch_size, num_heads, seq_len, depth)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

## 3-3) Masking

In [65]:
#숫자가 0인 부분 체크
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [66]:
#다음단어 가리기
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

## 3-4) One Layer

Multi-head attention은 남기고, encoder-decoder attention 제거

In [67]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  
  look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
  
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + inputs)

  # 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention)

  return tf.keras.Model(
      inputs=[inputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## 3-5) Multi Layer

- 포지셔널 인코딩에서 포지셔널 임베딩을 사용

In [79]:
# 포지셔널 임베딩 레이어
class PositionalEmbedding(tf.keras.layers.Layer):

  def __init__(self, max_position_embeddings, d_model):
    super(PositionalEmbedding, self).__init__()
    self.pos_embedding = tf.keras.layers.Embedding(input_dim=max_position_embeddings, output_dim=d_model)

  def call(self, inputs):
    input_shape = tf.shape(inputs)
    seq_length = input_shape[1]
    position_ids = tf.range(seq_length, dtype=tf.int32)[tf.newaxis, :]
    position_embeddings = self.pos_embedding(position_ids)
    return inputs + position_embeddings


In [80]:
import tensorflow as tf

def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  
  look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
 
    
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

   # 포지셔널 임베딩
  embeddings = PositionalEmbedding(max_position_embeddings, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## 3-6) GPT1 함수정의

In [81]:
def gpt1(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="gpt1"):
  
  inputs = tf.keras.Input(shape=(None,), name="inputs")
    
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)


  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, look_ahead_mask, dec_padding_mask])

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

# 4. 모델 생성 및 학습

## 4-1) 모델 생성

In [82]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 디코더의 층의 개수
D_MODEL = 256 #  입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = gpt1(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: tf.Tensor(
[[[0.6888994  0.5363984  0.50536674 ... 0.20925668 0.7225254  0.18461734]
  [0.41681027 0.7711873  0.55687183 ... 0.88645214 0.73807186 0.96231574]
  [0.74895    0.21239337 0.78249013 ... 0.77106947 0.5875087  0.6638409 ]
  ...
  [0.7975333  0.4263636  0.20540118 ... 0.5338001  0.39993593 0.943759  ]
  [0.8623851  0.546063   0.32038626 ... 0.3158327  0.9578311  0.8038364 ]
  [0.19556555 0.32256767 0.16199128 ... 0.11086425 0.45489293 0.410617  ]]], shape=(1, 10, 768), dtype=float32)